<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" /> 

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

## Implicit


## Implict transform
**当需要的时候就转换，否则啥也不干**


In [ ]:
case class Euro(value: Double)
case class Dollar(value: Double)

object Dollar{
    implicit def dollarToEuro(x: Dollar): Euro = Euro(x.value*0.75)
}
val a = Dollar(12)
val b: Euro = Dollar(10)

## implicit parameter
**隐式的定义一个参数，并且隐式的传递进去，对用户透明**


In [ ]:
object Test{
    def show(implicit h: Euro) = {
        println(h.value)
    }
}
implicit val h = Euro(12)
Test.show

## implicit Generate（隐式转换和隐式参数的协作）

**作用**：没有的时候生成一个，有了就什么也不干   
**区别**：隐式转换带参数，隐式生成不带参数

*隐式转换并没有输入参数，那么只要需要隐式的对象若没有声明，那么就生成一个，否则什么也不敢*
```scala
def hello(implicit h: ClassName) = {}
...
Test.hello //没有传递函数时会诱导隐式生成生成一个ClassName 对象给它
```


In [ ]:
case class ClassName(name: String)
object ClassName {
  //需要的时候生成一个 
  implicit def genWhenNeed: ClassName = ClassName("Example")
}
object Test{
    def hello(implicit h: ClassName) = {
        println(h.name)
    }
}
Test.hello(ClassName("jack"))
Test.hello

除了上面的那种方法诱导隐式生成发生以外

**还可以直接使用`implicitly[]`关键字**，注意是`[]`， 不是`()`

```scala
val b = implicitly[ClassName]
```

In [ ]:
val b = implicitly[ClassName]

## 隐式转换加宏

隐式转换加持宏可以和编译器打交道
使用中有两种方法：

- 方法一
```scala 
def getLineNumber(implicit line: LineNumber) ={
      println("add at ${line}"
      line
}
val s = getLineNumber
```

- 方法二
```scala
val s = implicitly[LineNumber]
```


(*注： Scala宏在Jupyter执行中会有问题，实例请在sbt环境下编译运行*)


In [ ]:
//获取当前行数
import language.experimental.macros
import reflect.macros.blackbox.Context

case class LineNumber(no: Int)

object LineNumber {
  implicit def genWhenNeed: LineNumber = macro Macros.lineNumberImpl
}

object Macros{
  def lineNumberImpl(c: Context): c.Expr[LineNumber] = {
    import c.universe._
    val lineNumber = c.enclosingPosition.line
    c.Expr[LineNumber](q"""${c.prefix}($lineNumber)""")
  }
}

In [ ]:
//获取变量名
import language.experimental.macros
import reflect.macros.blackbox.Context

case class SymbolName(name: String)
case class ClassName(name: String)

object SymbolName{
  implicit def genWhenNeed: SymbolName = macro Macros.symbolNameImpl
}
 

object Macros{
  def symbolNameImpl(c: Context): c.Expr[SymbolName] = {
    import c.universe._
    val symbolName = c.internal.enclosingOwner.name.decodedName.toString.trim
    c.Expr[SymbolName](q"""${c.prefix}($symbolName)""")
  }
}